In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gc

In [2]:
df = pd.read_csv("../andrews_dataset_subset.dat", dtype=np.float32)

In [3]:
arr = np.array(df)
del df
gc.collect()

42

In [ ]:
X = df.iloc[:, :676]
Y = df.iloc[:, -1]
N = len(X)
train_test_split = 0.95
N_X = int(train_test_split * N)
x_train = X[:N_X]
y_train = Y[:N_X]
x_test = X[N_X:]
y_test = Y[N_X:]

In [13]:
X = arr[:, :676]
Y = np.zeros((len(arr), 3))
Y[:, 0] = arr[:, -1] == 1.
Y[:, 1] = arr[:, -1] == 0.5
Y[:, 2] = arr[:, -1] == 0.

N = len(X)
train_test_split = 0.95
N_X = int(train_test_split * N)
x_train = X[:N_X]
y_train = Y[:N_X]
x_test = X[N_X:]
y_test = Y[N_X:]

In [14]:
inputs = tf.keras.Input(shape=(676,))
x = tf.keras.layers.Dense(16, activation = tf.nn.relu)(inputs)
#x = tf.keras.layers.Dense(16, activation = tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(3, activation = tf.nn.softmax)(x)
model = tf.keras.Model(inputs = inputs, outputs = outputs)

In [15]:
model.compile(loss=tf.keras.losses.MSE,metrics=["accuracy"] )

In [ ]:
def write_weights(model, file):
    with open(file, 'w') as f:
        weights = model.get_weights()
        for i in range(len(weights)):
            w = weights[i]
            if i % 2 == 0:
                layer_desc = "WEIGHTS"
                layer_type = "[f32;{}]".format(w.shape[0] * w.shape[1])
                s = "pub const LAYER_{}_WEIGHTS_SHAPE: (usize, usize) = ({},{});\n".format(i//2, w.shape[0],w.shape[1])
            else:
                layer_desc = "BIAS"
                layer_type = "[f32;{}]".format(len(w))
                s = ""

            s += "#[rustfmt::skip]\npub const LAYER_{}_{} : {} = ".format(i//2,layer_desc,layer_type)
            s += "[";
            for j in range(len(w)):
                if i % 2 == 0:
                    for k in range(len(w[j])):
                        s += str(w[j,k])
                        if k != len(w[j]) - 1:
                            s += ","
                else:
                    s += str(w[j])
                s += ","
            s += "];\n"
            f.write(s)

In [ ]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [ ]:
save = "train/model7"
import gc
import time
for i in range(100):
    gc.collect()
    time.sleep(5)
    #model.fit(x_train[:2_500_000], y_train[:2_500_000], epochs = 1)
    #model.fit(x_train[2_500_000:], y_train[2_500_000:], epochs = 1)
    #model.evaluate(x_test,y_test)
    model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=1)
    model.save(save+"_"+str(i)+".h5")
#write_weights(model, save+".weights")

38656/59375 [==================>...........] - ETA: 16s - loss: 0.1794 - accuracy: 0.5780

In [ ]:
write_weights(model, save+".weights")

In [ ]:
other_model = tf.keras.models.load_model("train/model4_7.h5")
other_model.evaluate(x_test, y_test)

In [ ]:
test = np.zeros((1, 676), dtype=np.float32)
other_model.predict(test)